In [104]:
from keplergl import KeplerGl
import pandas as pd
import requests

In [105]:
bahnhof = pd.read_csv("data/bahnhof.csv")
bahnhof.head()

,EVA_NR,DS100,NAME,Verkehr,longitude,latitude,state
0,8001944,TETN,Eutingen Nord,RV,8.753100,48.484700,Baden-Württemberg
1,8003074,MIA,Ingolstadt Audi,RV,11.407456,48.790496,Bayern
2,8001510,TDSA,Dornstetten-Aach,RV,8.482910,48.473300,Baden-Württemberg
3,8001966,MFOL,Feldolling,nur DPN,11.852244,47.895336,Bayern
4,8002535,EOBG,Halver-Oberbrügge,RV,7.574042,51.191867,Nordrhein-Westfalen


In [106]:
bahnhof[bahnhof["NAME"] == "Hamburg Hbf"]["DS100"].values[0]

'AH'

In [107]:
def search_route(start, end, time, time_type):
    #curl -X POST -H "Accept: application/json" -H "Content-Type: application/json" 
    #"https://openapi.trassenfinder.de/3.12/api/v5/routen/suche"  
    # -d '{"infrastruktur_id": 10, "sucheinstellungen": {"an_abzeit": "2022-11-13T16:38:36+02:00", "verkehrsart": "spfv_lok", "zeitvorgabe_typ": "abzeit"},  
    # "wegpunkte": [{"betriebsstelle": {"ds100": "HBS"}}, {"betriebsstelle": {"ds100":"MH"}}]}'
    start_ds100 = bahnhof[bahnhof["NAME"] == start]["DS100"].values[0]
    end_ds100 = bahnhof[bahnhof["NAME"] == end]["DS100"].values[0]

    headers = {'Accept': 'application/json', 'Content-Type': 'application/json'}
    data = {"infrastruktur_id": 10, "sucheinstellungen": {"an_abzeit": time, "verkehrsart": "spfv_lok", "zeitvorgabe_typ": time_type},  
    "wegpunkte": [{"betriebsstelle": {"ds100": start_ds100}}, {"betriebsstelle": {"ds100":end_ds100}}]}

    #print (data)

    r = requests.post('https://openapi.trassenfinder.de/3.12/api/v5/routen/suche', headers=headers, json=data)

    return r.json()

In [108]:
route = search_route("Braunschweig Hbf", "München Hbf", "2022-11-04T17:48:36+02:00", "abzeit")

In [109]:
source_lat = -1000
source_lng = -1000

target_lat = -1000
target_lng = -1000

content = ",".join(["source_lat", "source_lng", "target_lat", "target_lng"]) + "\n"

for s in route["result"]["gewichtete_route"]["routenpunkte"]:
    ds100 = s["ds100"]
    
    if source_lat == -1000:
        try:
            source_lat = bahnhof[bahnhof["DS100"] == ds100]["latitude"].values[0]
            source_lng = bahnhof[bahnhof["DS100"] == ds100]["longitude"].values[0]
        except:
            pass
    else:
        try:
            target_lat = bahnhof[bahnhof["DS100"] == ds100]["latitude"].values[0]
            target_lng = bahnhof[bahnhof["DS100"] == ds100]["longitude"].values[0]
            content += f"{source_lat},{source_lng},{target_lat},{target_lng}\n"
            source_lat = target_lat
            source_lng = target_lng
        except:
            pass

with open("data/data_route.csv", "w") as output:
    output.write(content)

In [110]:
route = pd.read_csv("data/data_route.csv")
route.head()

,source_lat,source_lng,target_lat,target_lng
0,52.252218,10.540295,52.257212,10.359760
1,52.257212,10.359760,52.318942,10.232145
2,52.318942,10.232145,52.335731,10.167325
3,52.335731,10.167325,52.354704,10.110601
4,52.354704,10.110601,52.376722,9.974599


In [115]:
config = {'version': 'v1',
 'config': {'visState': {
   'layers': [{
     'type': 'point',
     'config': {'dataId': 'bahnhof',
      'label': 'Point',
      'color': [18, 92, 119],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'lat': 'latitude', 'lng': 'longitude', 'altitude': None},
      'isVisible': True,
      'visConfig': {'radius': 10,
       'fixedRadius': False,
       'opacity': 0.09,
       'outline': False,
       'thickness': 2,
       'strokeColor': None,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radiusRange': [0, 50],
       'filled': True},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': {'name': 'state', 'type': 'string'},
      'colorScale': 'ordinal',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear'}},
    {
     'type': 'arc',
     'config': {'dataId': 'route',
      'label': 'source -> target arc',
      'color': [201, 0, 0],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'lat0': 'source_lat',
       'lng0': 'source_lng',
       'lat1': 'target_lat',
       'lng1': 'target_lng'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'thickness': 6.7,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'sizeRange': [0, 10],
       'targetColor': [155, 165, 46]},
      'hidden': False}}]}}}

In [116]:
map_1 = KeplerGl(height=800, data={"bahnhof": bahnhof, "route": route}, config=config)
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'layers': [{'type': 'point', 'config': {'dataId': 'b…

In [114]:
map_1.config

{'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': 'rsz12gf',
     'type': 'point',
     'config': {'dataId': 'bahnhof',
      'label': 'Point',
      'color': [18, 92, 119],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'lat': 'latitude', 'lng': 'longitude', 'altitude': None},
      'isVisible': True,
      'visConfig': {'radius': 10,
       'fixedRadius': False,
       'opacity': 0.09,
       'outline': False,
       'thickness': 2,
       'strokeColor': None,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
      